# Classification of Inappropriate/Offensive Text

## Import the essentials

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('dataset.csv')

## Data Preprocessing Part

In [2]:
# Function to remove unnecessary user tags from the entries
def clean_text(text):
    text = text.replace("@USER ", "")
    text = text.replace("@USER", "")
    return text

# Apply the cleaning function to the 'text' column
data['text'] = data['text'].apply(clean_text)

# Remove emojis from the samples
data.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

# Save the cleaned dataframe
data.to_csv('cleaned_dataset.csv', index=False)